# GPT

https://arxiv.org/abs/1706.03762

# self attention(SA)

1. 토큰 사이 중요도(연관성) 가중치를 계산하고 (쿼리-키 행렬 곱)
2. 가중치 반영 (가중치와 밸류 행렬 곱)

토큰끼리 가중치는 사실 원소 값이 다 달라야 하지만

일단은 똑같은 가중치(=평균)라고 가정하고 계산해봅시다

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 심플한 평균 가중치 구하기
데이터 `x` 예시

In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B,T,C)

print(x.shape)
x

torch.Size([4, 8, 2])


tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

`x` 내 timestep끼리 (토큰끼리) 가중치(평균) 계산

근데 auto-regressive로 할거라

가중치 첫번째 행은 1번째 토큰만 평균 -> 1

두번째 행은 1~2번째 토큰끼리 평균 -> 0.5 0.5

세번째 행은 1~3번째 토큰끼리 평균 -> 1/3 1/3 1/3

…

### lower triangular matrix 만들기

`torch.tril`: lower triangular matrix(하삼각행렬)을 만드는 메서드

In [29]:
tril = torch.tril(torch.ones(T,T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

### upper triangular 부분을 -inf로 마스킹

In [30]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

### softmax로 정규화

In [33]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [34]:
wei @ x

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [39]:
t = 1

x[0, :t+1], x[0, :t+1].mean(dim=0)

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152]]),
 tensor([-0.0894, -0.4926]))

## 평균이 아니라 가중치가 다르다면

In [49]:
tril = torch.tril(torch.ones(T,T))

q = torch.randn((T,C))
k = torch.randn((T,C))

wei = q @ k.T
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3530, 0.6470, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4054, 0.5365, 0.0580, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2387, 0.6678, 0.0713, 0.0222, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0750, 0.1745, 0.7418, 0.0012, 0.0075, 0.0000, 0.0000, 0.0000],
        [0.1437, 0.1235, 0.1103, 0.2730, 0.1915, 0.1580, 0.0000, 0.0000],
        [0.1410, 0.1429, 0.1199, 0.1503, 0.1610, 0.1463, 0.1386, 0.0000],
        [0.0325, 0.0137, 0.0257, 0.5486, 0.0565, 0.0408, 0.2649, 0.0173]])

In [50]:
wei @ x

tensor([[[ 0.1808, -0.0700],
         [-0.1688, -0.6168],
         [-0.0833, -0.5179],
         [-0.1312, -0.6247],
         [ 0.4188, -0.1372],
         [ 0.1670,  0.0404],
         [ 0.0859, -0.0478],
         [ 0.5525,  0.0294]],

        [[ 1.3488, -0.1396],
         [ 0.6611,  0.5751],
         [ 0.5819,  0.4898],
         [ 0.4006,  0.6595],
         [-1.3573,  0.5128],
         [ 0.2511, -0.0373],
         [ 0.1228, -0.1454],
         [ 0.9157,  0.0747]],

        [[-0.6631, -0.2513],
         [ 0.4194, -0.0101],
         [ 0.2823,  0.0292],
         [ 0.5020,  0.0954],
         [ 0.2389,  0.8363],
         [-0.2789, -0.3906],
         [-0.2724, -0.2151],
         [-0.9727, -0.1147]],

        [[ 1.6455, -0.8030],
         [ 1.4552, -0.4620],
         [ 1.3045, -0.3514],
         [ 1.2489, -0.2646],
         [-0.7472,  1.4394],
         [ 1.3987, -0.7167],
         [ 1.1210, -0.2242],
         [ 1.8984, -0.6285]]])

In [55]:
t = 1

x[0, :t+1], x[0, :t+1].mean(dim=0)

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152]]),
 tensor([-0.0894, -0.4926]))

## 어텐션 헤드 구현

scaled dot product attention 그림 참고

In [58]:
torch.manual_seed(1337)
B,T,C = 4, 8, 32 # batch time, channels
x = torch.randn(B,T,C)
x

tensor([[[ 0.1808, -0.0700, -0.3596,  ..., -0.8016,  1.5236,  2.5086],
         [-0.6631, -0.2513,  1.0101,  ...,  1.5333,  1.6097, -0.4032],
         [-0.8345,  0.5978, -0.0514,  ..., -0.4370, -1.0012, -0.4094],
         ...,
         [-0.8961,  0.0662, -0.0563,  ...,  2.1382,  0.5114,  1.2191],
         [ 0.1910, -0.3425,  1.7955,  ...,  0.3699, -0.5556, -0.3983],
         [-0.5819, -0.2208,  0.0135,  ..., -1.9079, -0.5276,  1.0807]],

        [[ 0.4562, -1.0917, -0.8207,  ...,  0.0512, -0.6576, -2.5729],
         [ 0.0210,  1.0060, -1.2492,  ...,  0.7859, -1.1501,  1.3132],
         [ 2.2007, -0.2195,  0.5427,  ..., -0.6445,  1.0834, -0.7995],
         ...,
         [ 0.3091,  1.1661, -2.1821,  ...,  0.6151,  0.6763,  0.6228],
         [ 0.0943, -0.3156,  0.7850,  ..., -1.5735,  1.3876,  0.7251],
         [ 0.6455, -0.3313, -1.0390,  ...,  0.0895, -0.3748, -0.4781]],

        [[-0.6067,  1.8328,  0.2931,  ...,  1.0041,  0.8656,  0.1688],
         [-0.2352, -0.2586,  0.0131,  ...,  0

In [67]:
# let's see a single head perform self-attention
head_dim = 16
query = nn.Linear(C, head_dim, bias=False)
key = nn.Linear(C, head_dim, bias=False)
value = nn.Linear(C, head_dim, bias=False)

In [69]:
q = query(x) # (B, T, 16)
k = key(x) # (B, T, 16)
v = value(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B,T,16) @ (B,16,T) ---> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0470, 0.9530, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1304, 0.2016, 0.6681, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0213, 0.0209, 0.5170, 0.4409, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6170, 0.0640, 0.1277, 0.0954, 0.0959, 0.0000, 0.0000, 0.0000],
         [0.3187, 0.3157, 0.0547, 0.0395, 0.0769, 0.1945, 0.0000, 0.0000],
         [0.0040, 0.0097, 0.0285, 0.2566, 0.0422, 0.5379, 0.1211, 0.0000],
         [0.0141, 0.0040, 0.0103, 0.1250, 0.0549, 0.0550, 0.0709, 0.6658]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0167, 0.9833, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3629, 0.5985, 0.0386, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0387, 0.0902, 0.2823, 0.5888, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1568, 0.0657, 0.5267, 0.0665, 0.1843, 0.0000, 0.0000, 0.0000],
         [0.0822, 0.013

In [70]:
out = wei @ v # (B,T,T) @ (B,T,16) ---> (B,T,16)
out.shape

torch.Size([4, 8, 16])

## note 1: 어텐션은 communication mechanism

timestep t까지 (1~t) 각 토큰끼리 

- 정보를 aggregation하고 (쿼리-키 행렬 곱)
- 가중합한 것임  (밸류 텐서에 행렬 곱으로 반영)

이번 강의에서는 글자단위 토크나이저 쓰니까

예를 들자면

- `쿼리`: 난 모음인데 나랑 있는 자음은 누구니
- `키`: 내가 자음이야
    - 해당 키를 가진 토큰이 가장 유사도 점수가 높음
- `밸류`: 실제 토큰에 쓰일 값
    - 여기다 가중합(행렬곱)으로 반영

해당 태스크에 필요한 정보들을 서로 문답하면서(communication)

토큰마다 최적의 hidden state를 찾는 것

## note 2: 어텐션에는 위치 정보(timestep)에 대한 정보가 없음

그래서 **positional encoding**이 필요함

어텐션은 그냥 임베딩 벡터 집합 내에 유사도 행렬을 구하고 밸류 텐서에 반영하는 것뿐이라 timestep 정보가 없음

## note 3: 각 미니 배치끼리는 communicate하지 않음

뭐… 당연한거죠?

셀프 어텐션은 각 미니 배치 내 토큰(timestep)끼리 계산

각 미니 배치는 독립적으로 수행 → GPU로 병렬처리 ㄱㄴ

## note 4: encoder block인 경우 마스킹(tril) 없음

학습할 때 얘기임 추론이 아니라 

디코더는 NLG(자연어 생성)가 원래 목적이라

다음 스텝에 올 토큰을 예측함

근데 다음 토큰에 관한 정보가 있으면 컨닝하는 거라 마스킹(tril)하는 거

인코더는 원래 목적이 NLU(자연어 이해)라서

셀프 어텐션할 때 모든 인풋 시퀀스를 고려함

마스킹이 필요없음

tril 적용 부분 주석 처리하면 됨

## note 5: cross-attention은 쿼리, 키/밸류의 소스가 다름

셀프 어텐션은 쿼리, 키, 밸류가 모두 같은 소스임

우리가 구현한 거 보면 모두 인풋 `x`로부터 `nn.Linear` 통과한 텐서들

하지만 크로스 어텐션인 경우 소스가 다름

예를 들어 `attention is all you need` 논문을 보면

인코더-디코더 모델을 쓰는데 (트랜스포머 아키텍처)
`쿼리`는 인풋 `x`가 소스고

`키/밸류`는 인코더 모듈 출력임

## note 6: 왜 쿼리-키 행렬곱한걸 sqrt(d_k)로 나눌까?

`attention is all you need` 논문 보면

어텐션 점수 구할 때 쿼리 키 행렬 곱을 스케일링함

$$
Attention=\text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}V\right)
$$

여기서 $d_k$는 헤드 내 텐서 차원 수 (=$d_{model}/\#\ of\ head$)

### 학습하기 전에는 틀린 가중치를 가짐

맨 처음 모델 initialization하면 학습되지 않은 상태

이때 추론시키면 정답이 아닌 다른 카테고리 확률이 높음 → loss 큼

마찬가지로 학습하기 전에 쿼리-키 행렬곱하면 

밸류에 대한 틀린 가중치를 갖게 됨

### 쿼리-키 행렬 곱하면서 분산이 커짐

$QK^T$하면서 대략 $d_k$만큼 분산이 커짐

In [71]:
torch.manual_seed(1337)
q = torch.randn(B, T, head_dim) # 4, 8, 16
k = torch.randn(B, T, head_dim)
wei = q @ k.transpose(-2, -1)

q.var(), k.var(), wei.var()

(tensor(1.0877), tensor(1.0594), tensor(18.4436))

wei의 분산이 18이나 커짐ㄷㄷ

분산이 크면 결국 softmax 취했을 때 균등하지 않고 극단적이게 됨 (원핫 벡터에 가까워짐)

예를 들어

In [72]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

분산을 키우면

In [73]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

학습 첫 스텝에서는 잘못된 추론으로

큰 loss를 가질 수밖에 없음

그러므로 kaiming initialization처럼 std gain을 보상해주자

In [74]:
torch.manual_seed(1337)
q = torch.randn(B, T, head_dim)
k = torch.randn(B, T, head_dim)
wei = q @ k.transpose(-2, -1) / head_dim**0.5

q.var(), k.var(), wei.var()

(tensor(1.0877), tensor(1.0594), tensor(1.1527))

# Layer normalization

배치 정규화랑 달리 각 row 방향으로 (피처 방향)으로 정규화

In [ ]:
class LayerNorm:
    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x- xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

# 데이터 살펴보기

mini shakespeare

In [2]:
with open('../mini_shakes.txt', 'r', encoding='utf8') as fp:
    text = fp.read()

print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


# 토큰화

이전에 해왔던 대로 그냥 한글자 단위로 vocab 설정

**토크나이저 vocab 사이즈** vs **timestep trade off**

vocab 사이즈가 작은 걸로 토큰화하면 time step(context window) 커집니다.

요즘은 영어인 경우 다 BPE나 titoken 써가지고 vocab size 좀 큽니다.

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


# train/val split

셰익스피어를 완벽히 암기하는게 아니라 셰익스피어같이 텍스트 생성하는 게 목표

오버피팅도 방지

In [6]:
n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]

# block size만큼 데이터 범위 설정

**block size: 학습할 때의 최대 context window**

(다음 토큰을 예측하기 직전의 토큰 수)

In [ ]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


블록 수만큼 학습

블록 크기가 8개라면 위처럼 8개의 학습 데이터가 만들어집니다.

이렇게 하면 모델이 매우 작은 컨텍스트(=1)부터 최대 컨텍스트(=8)까지 모두 학습할 수 있습니다.

→ 다양한 인풋에 적절한 답변 생성할 수 있게 됩니다. (인풋 길이 작아도, 커도 대답!)

이제 배치 4로 해서 만들어봅시다.

In [9]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

# loss 측정 함수

In [75]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [77]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384
n_head = 6
n_layer = 6
dropout = 0.2

# [optional] bigram model

In [88]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # targets.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [89]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [90]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [91]:
batch_size = 32

for steps in range(50000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps%5000==0:
        print('step', steps, 'loss:', loss.item())

step 0 loss: 4.729425430297852
step 5000 loss: 2.5123085975646973
step 10000 loss: 2.4576165676116943
step 15000 loss: 2.442767381668091
step 20000 loss: 2.4573700428009033
step 25000 loss: 2.4389965534210205
step 30000 loss: 2.4534378051757812
step 35000 loss: 2.4459445476531982
step 40000 loss: 2.463813543319702
step 45000 loss: 2.456265926361084


In [92]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


HAs; n og ARKEve tous oone ntanthintis n Thimed;
Houchy me t ckin by mapa, owis.
Andean she dva
Age f:

An t br'rtrercad locheres.
I oly I t onddorint biss be ceed vemmy I ngowhinothabld
tat hel I ompatyor yo t.
IIORIIN s, ir Bewilf sof ate k suser s y usprgr!
TUKiler igerindsolle CESIOFormegine s f pr's meany ghiele.
ORe man thethes smme t hen ffrend t whickitho.
f o n h s lewr ny atheeerklour, a


# GPTLanguageModel(nn.Module)
우리가 최종적으로 만들고 싶은 언어 생성 모델!

메서드 단위로 쪼개서 보자

## `def __init__(self)`

- `토큰 임베딩 테이블`
- `포지셔널 임베딩(인코딩) 테이블`
    - 원 논문에서는 sinusoidal로 했지만 여기서는 그냥 임베딩 모델로 함
- `트랜스포머 디코더 block들`
    - 여기 있는 Block은 뒤에서 만들도록 하겠음
- `layer normalization`
- `lm_head`

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.positional_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

## `def _init_weights(self, module)`

특이하게도 가중치 초기화 메서드를 따로 적용함

가중치의 경우 평균 0, std 0.02 정규분포로 초기화!

편향은 0으로 초기화

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.positional_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

        # better init
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

## `def forward(self, idx, targets)`

1. `토큰 임베딩`
2. `포지셔널 임베딩`도 더해서 추가
3. 모든 `트랜스포머 디코더 블록` 통과
4. `레이어 정규화`
5. `lm_head` 거쳐서 `logits` 계산
6. `targets`가 주어진 경우 `cross entropy loss` 계산


In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.positional_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

        # better init
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.positional_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

## `def generate(self, idx, max_new_tokens)`

주어진 `max_new_tokens` 만큼 계속 토큰 생성할 거임

1. `idx_cond`로 데이터가 `block_size` 만 있도록 남김
    1. 모델은 `block_size`만큼만 인풋으로 받을 수 있기 때문!
2. 모델 추론으로 `logits`, `loss` 구함
3. 다음 토큰 확률만 궁금하니 최종 timestep에 대한 `logits`만 남김
4. `F.softmax`로 다음 토큰 확률 구함
5. `torch.multinomial`로 다음 토큰 뽑음
6. 생성된 토큰 데이터에 `torch.cat`해서 끝에 붙임

반복 반복!

In [79]:
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.positional_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

        # better init
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.positional_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# class Block

핵심 부분을 만들자

## `def __init__(self, n_embed, n_head)`

그림에서 볼 수 있듯이

- `멀티 헤드 어텐션`
- `레이어 정규화`
- `피드포워드`

가 필요함

`head_size`는 가 헤드가 담당하는 차원 수!

In [80]:
class Block(nn.Module):
    """ Transformer block: communication follwed by computation """

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

## `def forward(self, x)`

그림처럼 `셀프 어텐션` 후 `피드포워드`에 넘기면 됨!

residual connection도 적용!

In [81]:
class Block(nn.Module):
    """ Transformer block: communication follwed by computation """

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

원 논문 그림을 보면

- `셀프 어텐션` 후에 `레이어 정규화`
- `피드포워드` 후에 `레이어 정규화`

이렇게 하는데 슨상님은 좋지 않다고 생각

배치 정규화랑 마찬가지로 `레이어 정규화`도

`행렬 곱`과 `활성화 함수` 때메 바뀌는 `std`를 미리 최적으로 잡아주는 용도임

그러므로 `행렬 곱`과 `활성화 함수` 전에 적용하는 것이 맞다고 생각

# class FeedForward

매우 간단

원 논문 식이랑 똑같이 감

$$
FFN(x) = max(0,\  xW_1 + b_1)W_2 + b2 
$$

In [82]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)

# class MultiHeadAttention

## `def __init__(self, num_heads, head_size)`

딱 그림처럼

- `여러 어텐션 헤드`
- `proejction layer`
- `드롭아웃`도 추가…

In [83]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size*num_heads, n_embed) # (n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

잘보면 어텐션 헤드는 `nn.Sequential`이 아니라 `ModuleList`임!

Sequential은 말그대로 순차적으로 `x1 → module1 → x2 → module2` 이렇게 가는거고

여기서는 순차적이 아닌 동시에 써서 `concat` 할거라 `nn.ModuleList` 씀

## `def forward(self, x)`

그림처럼

1. 모든 어텐션 헤드 결과값 `torch.cat`
2. `projection(Linear) layer` 통과

In [84]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size*num_heads, n_embed) # (n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# class Head

위에서 만든 헤드 쓰면 됨

- `쿼리, 키, 밸류`
- 마스킹을 위해 `tril`
    - `register_buffer`인 이유는 파라미터를 가진 게 아닌데 그냥 클래스 변수로 만들고 싶어서 그럼
- `드롭아웃`도 추가

In [85]:
class Head(nn.Module):
    """ one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # input size ---> (Batch, Timestep, Channels)
        B, T, C = x.shape

        q = self.query(x) # (B, T, head_size)
        k = self.key(x) # (B, T, head_size)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x) # (B, T, head_size)
        out = wei @ v # (B,T,T) @ (B,T,head_size) ---> (B,T, head_size)
        return out

# 학습

In [86]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


colab T4로 54분 걸렸습니다...

In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval==0 or iter==max_iters-1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# 생성

In [ ]:
# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))